# Mentoría 'de cómo clasificar en géneros a las canciones'
## Práctico II : Análisis de features de audio

Algunas aclaraciones:
- En 2) todos los pasos deberían poder hacerse en una línea, aunque si se complican y necesitan hacerlo en más no hay problema
- En 3) les dejé un buen spoiler para que lo completen
- Todas los conceptos y librerias que vamos a usar están documentados en el notebook
- Esta vez vamos a ser más rígidos con los plazos, la entrega es el 05/08, la única instancia previa de corrección son las entregas ANTES de la fecha mencionada, caso contrario es la nota final.

### Librerías

!pip3 install spotipy
!pip3 install pandas
!pip3 install spacy
!pip3 install pymusixmatch
!pip3 install nltk

# Agregar las librerías extra que se utilicen en esta celda y la siguiente

### Dependencias y acceso al API

In [4]:
import pandas as pd
import numpy as np
import spotipy
import spacy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
import json
import seaborn as sns
import tqdm

client_id = '46b333d567314a89a6254b6c6b054be6'
client_secret = '9d922c3613e441518349dcf55f7d5853'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

nlp = spacy.load("en_core_web_sm") #completar con el modelo que van a utilizar

sns.set_context(context='paper')

In [5]:
# aux methods

def song_url_for_request(artist, song_title):
    return "https://api.lyrics.ovh/v1/" + artist + '/' + song_title #str
    #example use
    #requests.get(song_url_for_request("Death Grips", "Hacker"))
    

def songs_from_album_id(album_id):
    songs = []
    album = sp.album(album_id)
    artist = album['artists'][0]['name']
    for item in album['tracks']['items']:
        track = {}
        track["song_name"] = item['name']
        track["lyrics"]=[]
        track["song_id"] = item['id']
        track["album_name"] = album['name']
        track["album_id"] = album["id"]
        track["artist"] = artist
        songs.append(track)
    return songs    #songs:List[dict]



def genres_by_artist_id(id): #id: str
    artist = sp.artist(id)
    genres = artist['genres']
    return genres    #genres: List[str]

In [6]:
genres_by_artist_id("5RADpgYLOuS2ZxDq7ggYYH")

['alternative hip hop',
 'escape room',
 'experimental hip hop',
 'hip hop',
 'industrial hip hop']

## Pregunta:
- Por qué Death Grips destaca respecto al género de hip hop? 
- Cómo se refleja su caracter experimental en sus letras?

### 2) Obtener datos para responder la pregunta
Una vez obtenidas las letras de las canciones, las procesaremos de la siguiente manera:
1. Tokenizar
2. Remover las [stop-words](https://es.wikipedia.org/wiki/Palabra_vac%C3%ADa)
3. [Lema](https://es.wikipedia.org/wiki/Lema_(ling%C3%BC%C3%ADstica))tizar las palabras que hayan quedado
4. Organizar todo en un DataFrame de la forma Canción-Artista-Letra donde Letra es el resultado de 3.

**Recomendación** : Usar un dataframe para su artista y otro para la competencia si es que eligierone esa opción, les va a ahorrar tiempo.

**Aclaración** : Si plantearon una pregunta que no requiera los datos de la consigna, obtengan esos datos.

First we build Death Grip's dataset. We'll be using lyrics from 5 albums;

- No Love Deep Web
- The money store
- The powers that B
- Bottomless Pit
- Year of the snitch

for a total of 70 songs. After sanitization, 69.

In [7]:
ALBUM_IDS = ["08aqY8lv4zx4uaqBUpMD8a", 
              "1PQDjdBpHPikAodJqjzm6a",
              "5Y04ylQjDWsawOUJXzY4YO",
              "4dIPUQHheyH9e6ioplvNT2", 
              "46eayJPxf1cBWWUqNa2MJJ"]

def lemmafy(doc):
    lemmas = []
    for token in doc:
        if not token.is_stop and token.is_alpha:
             lemmas.append(token.lemma_)
    return lemmas

In [8]:
#arm base dataset
def build_from_album_ids(album_id_array):
    all_songs = []
    for album in tqdm.tqdm(ALBUM_IDS):
        all_songs += songs_from_album_id(album)
    return all_songs
    
all_songs = build_from_album_ids(ALBUM_IDS)

100%|██████████| 5/5 [00:01<00:00,  3.60it/s]


In [9]:
#put song lyrics in dataset
def add_lyrics_to_data(all_songs):
    for song in tqdm.tqdm(all_songs):
        response = requests.get(song_url_for_request(song["artist"],song["song_name"]))
        json_data = json.loads(response.content)
        try:
            lyrics_raw = json_data["lyrics"]
            doc = nlp(lyrics_raw)
            lemmed = lemmafy(doc)
            song["lyrics"]=lemmed
        except: #found a song without lyrics!
            print(song["song_name"])
            song["lyrics"]=None
    return all_songs

all_songs = add_lyrics_to_data(all_songs)


 44%|████▍     | 31/70 [00:40<00:50,  1.29s/it]

Have A Sad Cum BB


100%|██████████| 70/70 [01:31<00:00,  1.30s/it]


In [10]:
dg_songs_base = pd.DataFrame(all_songs)

In [11]:
# sanitize
bool_series = pd.notnull(dg_songs_base["lyrics"])
dg_songs_base[bool_series].head(2)

,album_id,album_name,artist,lyrics,song_id,song_name
0,08aqY8lv4zx4uaqBUpMD8a,No Love Deep Web,Death Grips,"[stone, wall, dog, gaze, duct, tape, ceiling, ...",0ynz4gps9rvFEsz1bz43Iz,Come Up And Get Me
1,08aqY8lv4zx4uaqBUpMD8a,No Love Deep Web,Death Grips,"[burn, burn, takin, turn, turn, know, burn, bu...",6fab5rAf9v58GR8P2Q1o1s,Lil Boy


Como vamos a compararlo con otros 

### 3) Encontrar los n-gramas (n= 1, 2) más comunes y usarlos en gráficos.
Utilizando las librerías de la celda de abajo obtendremos los conjuntos de 1 y 2 palabras más comunes para cada artista con el que trabajaremos. Luego usaremos los unigramas para graficar una [word-cloud](https://i.imgur.com/8I8aJ1N.png) y un gráfico de distribución de frecuencia para unigramas y bigramas por autor (o de la forma que se adapte a su pregunta).

In [12]:
from collections import Counter
from nltk import ngrams
count = Counter()
for lyric in tqdm.tqdm(dg_songs_base["lyrics"].values):
    thing = ngrams(lyric,1)
    try:
        result = Counter(thing)
    except:
        print(thing)
    #for item, count in sorted(result.items()):
    #    if count>=10:
    #        print(item,count)
result.most_common(10)

100%|██████████| 70/70 [00:00<00:00, 11080.97it/s]

<generator object ngrams at 0x7f266f463f68>


[(('wah',), 84),
 (('disappoint',), 13),
 (('Wah',), 12),
 (('chin',), 3),
 (('hold',), 3),
 (('job',), 3),
 (('fuck',), 3),
 (('tight',), 3),
 (('shot',), 2),
 (('life',), 2)]

In [132]:
import itertools
lyrics = list(itertools.chain(*dg_songs_base["lyrics"].head(30).values))
len(lyrics)

5138

In [133]:
from collections import Counter
from nltk import ngrams, bigrams
import nltk
uni_tokens = ngrams(lyrics,1)
bi_tokens = bigrams(lyrics)
bi_counts = Counter(bi_tokens)
uni_counts = Counter(uni_tokens)
#print([(item, counts.count(item)) for item in sorted(set(bi_tokens))])
fdist = nltk.FreqDist(bi_tokens)
for k,v in fdist.items():
    print(k,v)
#print(dg_songs_base["lyrics"].head(2))
bi_counts.most_common()

[(('lil', 'boy'), 19),
 (('be', 'not'), 17),
 (('black', 'quarterback'), 13),
 (('people', 'people'), 11),
 (('wanna', 'know'), 10),
 (('real', 'shit'), 9),
 (('pop', 'pop'), 9),
 (('watch', 'watch'), 9),
 (('kill', 'like'), 9),
 (('like', 'ya'), 9),
 (('ya', 'hate'), 9),
 (('quarterback', 'black'), 9),
 (('shit', 'go'), 8),
 (('hey', 'kid'), 8),
 (('death', 'west'), 7),
 (('oh', 'oh'), 7),
 (('see', 'footage'), 7),
 (('like', 'know'), 7),
 (('smokin', 'rock'), 7),
 (('rock', 'real'), 7),
 (('shit', 'people'), 7),
 (('shit', 'bout'), 6),
 (('burn', 'burn'), 6),
 (('takin', 'turn'), 6),
 (('turn', 'turn'), 6),
 (('fit', 'ta'), 6),
 (('blood', 'flow'), 6),
 (('artificial', 'death'), 6),
 (('west', 'east'), 6),
 (('east', 'pyramid'), 6),
 (('pyramid', 'deck'), 6),
 (('deck', 'shine'), 6),
 (('know', 'know'), 6),
 (('Hustle', 'bone'), 6),
 (('bone', 'comin'), 6),
 (('comin', 'mouth'), 6),
 (('blower', 'system'), 6),
 (('hate', 'kill'), 6),
 (('-PRON-', '-PRON-'), 6),
 (('area', 'area'), 6)

### 4) Escribir un informe

Este informe tiene que describir qué datos que obtuvieron, el volumen de estos (# de canciones del artista, # de canciones de la competencia, etc), una explicación de cómo estos se relacionan con la pregunta planteada, cómo ayudan a responderla y los resultados que obtuvieron. Usar la celda de abajo con formato markdown.


